In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from ipywidgets import FileUpload, VBox, FloatSlider, SelectMultiple, Button, Output
from IPython.display import display, clear_output

In [7]:
pip install nbformat>=4.2.0


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
df= pd.read_excel('Book1.xlsx')

In [3]:
# Create widgets for criteria selection and weighting
criteria_output = Output()

def create_criteria_widgets():
    if 'df' not in globals():
        with criteria_output:
            print("Please upload a file first")
        return
    
    # Create criteria selection widget
    criteria_select = SelectMultiple(
        options=df.columns.tolist(),
        description='Select Criteria',
        style={'description_width': 'initial'}
    )
    
    # Create weight sliders container
    weight_sliders = VBox([])
    
    def on_criteria_change(change):
        # Clear existing sliders
        weight_sliders.children = []
        
        # Create new sliders for selected criteria
        for criterion in change['new']:
            slider = FloatSlider(
                value=1.0,
                min=0.0,
                max=5.0,
                step=0.1,
                description=criterion,
                style={'description_width': 'initial'}
            )
            weight_sliders.children += (slider,)
    
    criteria_select.observe(on_criteria_change, names='value')
    
    # Create calculate button
    calculate_button = Button(description='Calculate Scores')
    
    def on_calculate_click(b):
        if not criteria_select.value:
            print("Please select at least one criterion")
            return
            
        # Calculate weighted scores
        total_weight = sum(slider.value for slider in weight_sliders.children)
        weighted_scores = pd.Series(0.0, index=df.index)
        
        for criterion, slider in zip(criteria_select.value, weight_sliders.children):
            # Normalize the criterion scores
            normalized_scores = (df[criterion] - df[criterion].min()) / (df[criterion].max() - df[criterion].min())
            weighted_scores += normalized_scores * (slider.value / total_weight)
        
        # Add scores to dataframe
        df['Composite_Score'] = weighted_scores
        
        # Display results
        print("\nTop 10 Companies by Score:")
        display(df.nlargest(10, 'Composite_Score')[['Composite_Score'] + list(criteria_select.value)])
        
        # Create score distribution plot
        fig = px.histogram(df, x='Composite_Score',
                          title='Distribution of Composite Scores')
        fig.show()
    
    calculate_button.on_click(on_calculate_click)
    
    # Display all widgets
    display(criteria_select)
    display(weight_sliders)
    display(calculate_button)

with criteria_output:
    create_criteria_widgets()

display(criteria_output)

Output()


Top 10 Companies by Score:


,Composite_Score,score_1,score_2,score_3
0,0.75,1,0,1
2,0.75,1,1,0
1,0.25,0,1,0


In [18]:
df

,short_name,score_1,score_2,score_3,sector,Composite_Score
0,AAA,1,0,1,IT,0.75
1,BBB,0,1,0,Energy,0.25
2,CCC,1,1,0,IT,0.75


In [22]:


# Create output widget for displaying results
percentile_output = Output()

# Create percentile slider
percentile_slider = FloatSlider(
    value=50,
    min=0,
    max=100,
    step=1,
    description='Percentile Threshold:',
    style={'description_width': 'initial'}
)

def on_percentile_change(change):
    with percentile_output:
        percentile_output.clear_output()
        
        # Calculate threshold value
        threshold = df['Composite_Score'].quantile(change['new'] / 100)
        
        # Filter dataframe
        global df_sl
        df_sl = df[df['Composite_Score'] >= threshold].copy()
        
        # Display number of companies
        print(f"Number of companies above {change['new']}th percentile: {len(df_sl)}")
        
        # Create sector composition pie chart
        sector_counts = df_sl['sector'].value_counts()
        fig = px.pie(values=sector_counts.values, 
                    names=sector_counts.index,
                    title=f'Sector Composition (Companies above {change["new"]}th percentile)')
        fig.show()
        
        # Display filtered dataframe
        print("\nTop filtered Companies:")
        display(df_sl.iloc[:10])

# Link the slider to the update function
percentile_slider.observe(on_percentile_change, names='value')

# Display widgets
display(percentile_slider)
display(percentile_output)

# Trigger initial update
on_percentile_change({'new': percentile_slider.value})
 

FloatSlider(value=50.0, description='Percentile Threshold:', step=1.0, style=SliderStyle(description_width='in…

Output()